In [81]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import os

In [39]:
pandasDataFrame = pd.read_excel(path_to_dataset)
print(pandasDataFrame.describe())

      AT      V       AP     RH      PE
0  14.96  41.76  1024.07  73.17  463.26
1  25.18  62.96  1020.04  59.08  444.37
2   5.11  39.40  1012.16  92.14  488.56
3  20.86  57.32  1010.24  76.64  446.48
4  10.82  37.50  1009.23  96.62  473.90
5  26.27  59.44  1012.23  58.77  443.67
6  15.89  43.96  1014.02  75.24  467.35
7   9.48  44.71  1019.12  66.43  478.42
8  14.64  45.00  1021.78  41.25  475.98
9  11.74  43.56  1015.14  70.72  477.50
                AT            V           AP           RH           PE
count  9568.000000  9568.000000  9568.000000  9568.000000  9568.000000
mean     19.651231    54.305804  1013.259078    73.308978   454.365009
std       7.452473    12.707893     5.938784    14.600269    17.066995
min       1.810000    25.360000   992.890000    25.560000   420.260000
25%      13.510000    41.740000  1009.100000    63.327500   439.750000
50%      20.345000    52.080000  1012.940000    74.975000   451.550000
75%      25.720000    66.540000  1017.260000    84.830000   468

#### Convert pandas DataFrame to numpy array

In [41]:
numpyArray = pandasDataFrame.to_numpy()
print(numpyArray.shape)

(9568, 5)


#### Create input and output

In [50]:
inputArray = numpyArray[:, :4]
outputArray = numpyArray[:, 4:5]

#### Split the dataset into training and testing sets

In [36]:
X_train, X_test, y_train, y_test = train_test_split(inputArray, outputArray, test_size=0.2, random_state=42)

#### Standardize the features

In [52]:
scaler_x = StandardScaler()
X_train = scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test)

#### Standardize the targets

In [7]:
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

#### Define the dataset class (the purpose of the class is to convert datasets into tensors, and define len and getitem functions)

In [72]:
class PowerPredictionDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

#### Create DataLoader instances

In [73]:
train_dataset = PowerPredictionDataset(X_train, y_train)
test_dataset = PowerPredictionDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

#### Define neural network model

In [74]:
class PowerPredictionModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(PowerPredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)       
        self.fc2 = nn.Linear(hidden_size, 1)              
        
        
    def forward(self, input_tensor):
        hidden_layer_output = torch.nn.functional.relu(self.fc1(input_tensor))
        final_layer_ouput = self.fc2(hidden_layer_output)
        return final_layer_ouput
    

#### Instantiate the model, loss function, and optimizer

In [75]:
input_size = X_train.shape[1]
model = PowerPredictionModel(input_size, 5)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

#### Create training loop

In [78]:
number_epochs = 20

for epoch in range(number_epochs):
    model.train()
    epoch_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    average_epoch_loss = epoch_loss / len(train_loader)    
    print(f"Epoch: {epoch}  => => => => => => =>  Loss: {average_epoch_loss}")

Epoch: 0  => => => => => => =>  Loss: 440.8221073150635
Epoch: 1  => => => => => => =>  Loss: 345.6162550608317
Epoch: 2  => => => => => => =>  Loss: 266.50255692799885
Epoch: 3  => => => => => => =>  Loss: 200.62731329600015
Epoch: 4  => => => => => => =>  Loss: 147.48871901830037
Epoch: 5  => => => => => => =>  Loss: 106.69346625010172
Epoch: 6  => => => => => => =>  Loss: 77.69152844746908
Epoch: 7  => => => => => => =>  Loss: 57.730156342188515
Epoch: 8  => => => => => => =>  Loss: 44.646056747436525
Epoch: 9  => => => => => => =>  Loss: 36.34996116956075
Epoch: 10  => => => => => => =>  Loss: 31.022942217191062
Epoch: 11  => => => => => => =>  Loss: 27.398744050661723
Epoch: 12  => => => => => => =>  Loss: 25.178469530741374
Epoch: 13  => => => => => => =>  Loss: 23.74189244111379
Epoch: 14  => => => => => => =>  Loss: 22.639245986938477
Epoch: 15  => => => => => => =>  Loss: 22.078489581743877
Epoch: 16  => => => => => => =>  Loss: 21.570253245035808
Epoch: 17  => => => => => => 

####  Evaluate the model on the rest set

In [80]:
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_loader:
        outputs = model(inputs)
        test_loss += criterion(outputs, targets).item()
        
    average_test_loss = test_loss / len(test_loader)
    print(f"Average test loss:  {average_test_loss}")

Average test loss:  20.492122840881347
